In [ ]:
#default_exp extend_pandas

# Extend pandas 
This is a collections of functions to add some additional functionality to pandas

In [ ]:
#export
import pandas as pd
import pyodbc
import numpy as np
import os
import psutil
import gc

In [ ]:
#export
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
#export
def evaluate_join(df1, df2, left_on, right_on, output=False):
    print('DF1: ', df1.shape)
    print('DF2: ', df2.shape)
    df1 = df1[left_on]
    is_pkey1 = df1.shape == df1.drop_duplicates().shape
    print('DF1 is joining on primary key: ', is_pkey1)
    df2 = df2[right_on]
    is_pkey2 = df2.shape == df2.drop_duplicates().shape
    print('DF2 is joining on primary key: ', is_pkey2)
    dfjoin = df1.merge(df2, how='outer', left_on = left_on, right_on = right_on, indicator = True)
    
    both = dfjoin.loc[dfjoin['_merge'] == 'both'].shape[0]
    left = dfjoin.loc[dfjoin['_merge'] == 'left_only'].shape[0]
    left_both = dfjoin.loc[(dfjoin['_merge'] == 'left_only') | (dfjoin['_merge'] == 'both')].shape[0]
    right = dfjoin.loc[dfjoin['_merge'] == 'right_only'].shape[0]
    full = dfjoin.shape[0]
    print('\n')
    print('Join Type:\t\t      Rows \t %')
    print('Both \t\t\t {0:10d} \t ({1:2.5%})'.format(both, both/full) )
    print('Left \t\t\t {0:10d} \t ({1:2.5%})'.format(left, left/full) )
    print('Left + Both \t\t {0:10d} \t ({1:2.5%})'.format(left_both, left_both/full) )
    print('Right \t\t\t {0:10d} \t ({1:2.5%})'.format(right, right/full) )
    print('Left + Both + Right \t {0:10d} \t ({1:2.0%})'.format(full, full/full) )
    if output:
        return dfjoin

In [ ]:
#export
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
#export
def optimize_dataframe_memory_usage(df):

    print('Original DataFrame MemoryUsage: ' + mem_usage(df))
    
    # Create copy for temp usage
    optimized_df = df.copy()
    
    # Downcast all int types
    df_int = optimized_df.select_dtypes(include=['int', 'int64'])
    converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
    
    # Downcast all float type
    df_float = optimized_df.select_dtypes(include=['float', 'float64'])
    converted_float = df_float.apply(pd.to_numeric,downcast='float')
    
    optimized_df[converted_int.columns] = converted_int
    optimized_df[converted_float.columns] = converted_float
    
    # Select all object column types
    df_obj = optimized_df.select_dtypes(include=['object'])

    # Convert object type to category type if unique values < 0.5 to converted_obj
    for col in df_obj.columns:
        num_unique_values = len(df_obj[col].unique())
        num_total_values = len(df_obj[col])
        if num_unique_values / num_total_values < 0.5:
            optimized_df[col] = df_obj[col].astype('category')
        else:
            optimized_df[col] = df_obj[col]

    del df_int, converted_int, df_float, converted_float, df_obj, df
    gc.collect()
    df_int = pd.DataFrame()
    converted_int = pd.DataFrame()
    df_float = pd.DataFrame()
    converted_float = pd.DataFrame()
    df_obj = pd.DataFrame()
    df = pd.DataFrame()
    
    print('Optimized DataFrame MemoryUsage: '+ mem_usage(optimized_df))
    
    return optimized_df    

In [ ]:
#export
def zero_pad(number, width):
    return str(number).zfill(width)    

In [ ]:
df = pd.DataFrame({'num':[5]})
df

,num
0,5


In [ ]:
df['num_as_str_padded'] = df['num'].apply(lambda x : zero_pad(x,2))
df

,num,num_as_str_padded
0,5,05
